<a href="https://colab.research.google.com/github/ZeynepOztek/Python-reniyorum/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
import pickle
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForTokenClassification, pipeline

# Gerekli kütüphane ve modellerin import edilmesi

data = pd.read_csv('/content/data.csv', encoding='utf-8')  # Data setinin projeye eklenmesi

# Veri temizleme: NaN değerleri kontrol et ve kaldır
data = data.dropna()  # NaN içeren satırları kaldır

# Özellikler (X) ve etiketler (y) olarak ayır
X = data['cümle']  # Yorumlar (cümle sütununu alıyoruz)
y = data[['entity', 'sentiment']]  # Pozitif, Negatif, Nötr etiketlerinin olduğu sütunlar

# TF-IDF vektörleştirici oluştur
tfidf_vectorizer = TfidfVectorizer()
X_vectorized = tfidf_vectorizer.fit_transform(X.values.astype('U'))  # Yorumları TF-IDF özelliklerine dönüştür

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# MultiOutputClassifier ile LightGBM modelini oluştur
model = MultiOutputClassifier(LGBMClassifier())
model.fit(X_train, y_train)  # Modeli eğitim verisi ile eğit

# Veriyi pickle kullanarak dosyaya kaydet
with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)

# Veriyi dosyadan geri yükleme
with open('data.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

print(loaded_data)  # Yüklenen veriyi yazdır

# TF-IDF vektörleştiriciyi pickle kullanarak dosyaya kaydet
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

# Modeli pickle kullanarak dosyaya kaydet
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Test seti için tahminlerde bulun
y_pred = model.predict(X_test)

# Her bir etiket için sınıflandırma raporunu yazdır
for i, column in enumerate(y.columns):
    print(f"Label: {column}")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("\n")

# Her etiket için doğruluk skorlarını yazdır
print("Overall accuracy scores for each label:")
for i, column in enumerate(y.columns):
    score = accuracy_score(y_test[column], y_pred[:, i])
    print(f"{column}: {score:.4f}")

# Test için
comment = "Turkcell 229 TL Lik ücret yansıtmış faturama. Ne olduğu belli değil. Mesajda yazan ibare şu:HTP click n Play servisinden 04.08. 2024 10:17:29 Tarihinde 229 TL'lik Paycell Mobil ödeme işleminiz gerçekleşmiştir. Bu harcamanızla tam 458 Paycell puan kazandınız. Paycell Puanlarınızı ister internet ister dilediğiniz sektörde geçerli indirim kodu olarak kullanabilirsiniz. Puanlarınızı hemen görmek için; paycell.com.tr/app B002. Bir de abonelik diyor daha sonraki ayda çekecek yani. Uygulamada falan da gözükmüyor. Ne olduğu nasıl iptal edileceği."
comment_vectorized = tfidf_vectorizer.transform([comment])
res = model.predict(comment_vectorized)

# Sonuçları 'Evet' ve 'Hayır' olarak yorumla
tahmin_list = []
for i, column in enumerate(y.columns):
    tahmin = res[0][i]
    if tahmin == 'pozitif':
        tahmin_list.append(f"{column}: Evet")
    elif tahmin == 'nötr':
        tahmin_list.append(f"{column}: 0")
    else:
        tahmin_list.append(f"{column}: Hayır")

# Tahminleri yazdır
for tahmin in tahmin_list:
    print(tahmin)

# BERT NER modeli kullanımı
tokenizer = BertTokenizer.from_pretrained('saribasmetehan/bert-base-turkish-uncased-ner')
model_ner = BertForTokenClassification.from_pretrained('saribasmetehan/bert-base-turkish-uncased-ner')

# NER pipeline
ner = pipeline("ner", model=model_ner, tokenizer=tokenizer)

# Örnek bir cümle üzerinde NER işlemi
ner_results = ner(comment)
print(ner_results)


In [ ]:
comment = "Turkcell 229 TL Lik ücret yansıtmış faturama. Ne olduğu belli değil. Mesajda yazan ibare şu:HTP click n Play servisinden 04.08. 2024 10:17:29 Tarihinde 229 TL'lik Paycell Mobil ödeme işleminiz gerçekleşmiştir. Bu harcamanızla tam 458 Paycell puan kazandınız. Paycell Puanlarınızı ister internet ister dilediğiniz sektörde geçerli indirim kodu olarak kullanabilirsiniz. Puanlarınızı hemen görmek için; paycell.com.tr/app B002. Bir de abonelik diyor daha sonraki ayda çekecek yani. Uygulamada falan da gözükmüyor. Ne olduğu nasıl iptal edileceği."
comment_vectorized = tfidf_vectorizer.transform([comment])
res = model.predict(comment_vectorized)

# Sonuçları 'Evet' ve 'Hayır' olarak yorumla
tahmin_list = []
for i, column in enumerate(res):
    tahmin = res[i]
    result = {
    'entity': tahmin[0],
    'sentiment': tahmin[1].strip().strip('"')
}


In [ ]:
result

In [139]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from transformers import BertTokenizer, BertForTokenClassification, pipeline

app = FastAPI()

# Model ve vektörleştirici dosyalarını yükleme
with open('vectorizer.pkl', 'rb') as file:
    tfidf_vectorizer = pickle.load(file)

with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# BERT NER modeli yükleme
tokenizer = BertTokenizer.from_pretrained('saribasmetehan/bert-base-turkish-uncased-ner')
model_ner = BertForTokenClassification.from_pretrained('saribasmetehan/bert-base-turkish-uncased-ner')
ner = pipeline("ner", model=model_ner, tokenizer=tokenizer)

# Gelen JSON isteklerinin yapısını tanımlamak için BaseModel kullanımı
class TextInput(BaseModel):
    text: str

# API'nin kök (root) endpoint'i, basit bir hoş geldiniz mesajı döner
@app.get("/")
def read_root():
    return {"message": "Welcome to the Text Classification and NER API"}

# Tahmin ve NER işlemi yapan endpoint
@app.post("/predict")
def predict(text_input: TextInput):
    try:
        # Gelen metni TF-IDF ile vektörleştir
        comment_vectorized = tfidf_vectorizer.transform([text_input.text])

        # Vektörleştirilmiş metinle modelden tahmin al
        res = model.predict(comment_vectorized)

        # Tahmin sonuçlarını işleme
        tahmin_list = {}
        for i, column in enumerate(['entity', 'sentiment']):
            tahmin = res[i]
            result = {'entity': tahmin[0]}  # Tahmin edilen entity
            sentiment = tahmin[1].strip().strip('"')  # Tahmin edilen sentiment

            # Sentiment'i Türkçe'ye çevirme
            if sentiment == 'pozitif':
                result['sentiment'] = 'olumlu'
            elif sentiment == 'nötr':
                result['sentiment'] = 'nötr'
            else:
                result['sentiment'] = 'olumsuz'

            # Entity'lerin listesini oluşturma
            entity_list = [item['entity'] for item in tahmin_list]

        # Sonuçları döndürme
        return {"results": tahmin_list, 'entity_list': entity_list}

    # Herhangi bir hata durumunda HTTP 500 hatası döndürme
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [147]:
from sklearn.metrics import classification_report, accuracy_score

# Test verisini yükleyin
test_data = pd.read_csv('/content/test.csv', encoding='utf-8')

# Test verisini temizleme: NaN değerleri kontrol et ve kaldır
test_data = test_data.dropna()

# Özellikler (X_test) ve etiketler (y_test) olarak ayır
X_test = test_data['cümle']
y_test = test_data[['entity', 'sentiment']]

# Test verilerini TF-IDF ile dönüştür
X_test_vectorized = tfidf_vectorizer.transform(X_test.values.astype('U'))

# Model ile tahmin yapma
y_pred = model.predict(X_test_vectorized)

# Her bir etiket için sınıflandırma raporunu yazdır
for i, column in enumerate(y_test.columns):
    print(f"Label: {column}")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("\n")

# Her etiket için doğruluk skorlarını yazdır
print("Overall accuracy scores for each label:")
for i, column in enumerate(y_test.columns):
    score = accuracy_score(y_test[column], y_pred[:, i])
    print(f"{column}: {score:.4f}")


Label: entity
                                          precision    recall  f1-score   support

                                "650 TL"       1.00      1.00      1.00         1
             "Birleşik Demokratik Parti"       0.00      0.00      0.00         1
                                 "Fairy"       1.00      1.00      1.00         1
                               "Telefon"       0.50      1.00      0.67         1
                           "altın rengi"       0.00      0.00      0.00         1
                              Afganistan       0.00      0.00      0.00         1
                            Akbulut Otel       0.00      0.00      0.00         1
                        Altın Küre Ödülü       0.00      0.00      0.00         1
                                 Andorra       0.00      0.00      0.00         1
                    Android, PC, telefon       0.00      0.00      0.00         1
             Apple Store, Macbook Pro M3       1.00      1.00      1.00         1
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [148]:
import nest_asyncio
import asyncio
from aiohttp import web

nest_asyncio.apply()

In [149]:
from pyngrok import ngrok
auth_token = "2kPG2JmX4ZOk0nElyMX57rExFzK_ny22Ra6XN2McL5Q9xgtZ"

!ngrok authtoken $auth_token
ngrok_tunnel = ngrok.connect(8000)
ngrok_tunnel.public_url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


'https://24f9-35-234-14-57.ngrok-free.app'

In [ ]:
import uvicorn
from fastapi import FastAPI
uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [47364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     35.245.31.63:0 - "POST / HTTP/1.1" 405 Method Not Allowed
INFO:     35.245.31.63:0 - "POST / HTTP/1.1" 405 Method Not Allowed
INFO:     35.245.31.63:0 - "POST / HTTP/1.1" 405 Method Not Allowed
